In [2]:
import duckdb
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

In [1]:
def encode_data(df, categories):
    encoders = {}
    for col, cats in categories.items():
        df[col] = pd.Categorical(df[col], categories=cats)
        encoders[col] = OneHotEncoder(categories=[cats], drop=None, sparse=False)
        encoded = encoders[col].fit_transform(df[[col]])
        for i, category in enumerate(cats):
            df[f"{col}_{category}"] = encoded[:, i]
        df.drop(columns=[col], inplace=True)
    return df, encoders

In [3]:
full_transactions_file = 'data/transaction_dataset.parquet'
full_accounts_file = 'data/account_dataset.parquet'
train_target_file = 'data/train_target_dataset.parquet'

In [4]:
# sample code
duckdb.sql(f"""
           SELECT txn.month, acc.assigned_bank, COUNT(txn.*) AS num_fraud_txn
           FROM '{full_transactions_file}' txn
             JOIN '{train_target_file}' launder ON launder.transaction_id = txn.transaction_id
             JOIN '{full_accounts_file}' acc ON txn.account_id = acc.account_id AND txn.transaction_direction = 'inbound'
           GROUP BY txn.month, acc.assigned_bank
           """).df()

,month,assigned_bank,num_fraud_txn
0,9,large_digital_1,134
1,7,large_1,633
2,6,large_1,559
3,9,local_3,175
4,4,large_2,331
...,...,...,...
82,7,local_1,237
83,5,small_digital_2,8
84,10,large_digital_1,39
85,1,small_digital_1,14


# Split data by banks

In [5]:
# get list of banks
banks = duckdb.sql(f"""
    SELECT DISTINCT assigned_bank
    FROM '{full_transactions_file}' txn
        JOIN '{full_accounts_file}' acc ON txn.account_id = acc.account_id AND txn.transaction_direction = 'inbound'
""").df()['assigned_bank'].tolist()

bank_txns = []
for bank in banks:
    df_temp = duckdb.sql(f"""
                SELECT txn.*,
                         CASE WHEN txn.transaction_id IN (SELECT transaction_id FROM '{train_target_file}') THEN 1 ELSE 0 END AS IsFraud
                FROM  '{full_transactions_file}' txn
                    JOIN '{full_accounts_file}' acc ON txn.account_id = acc.account_id AND txn.transaction_direction = 'inbound'
                WHERE acc.assigned_bank = '{bank}'
               """).df()
    bank_txns.append(df_temp)

In [13]:
def train_model(df, target:str, inputs:list):
    X = df[inputs].values
    y = df[target].values

    dmatrix = xgb.DMatrix(X, label=y)
    params = {'objective': 'binary:logistic', 'max_depth': 3, 'learning_rate': 0.1}
    model = xgb.train(params, dmatrix, num_boost_round=10)
    return model

In [14]:
model = train_model(bank_txns[0], 'IsFraud', ['amount'])

In [17]:
model.get_dump(with_stats=True)

['0:[f0<568] yes=1,no=2,missing=2,gain=0.625,cover=533641.062\n\t1:leaf=-0.11347165,cover=248556.391\n\t2:leaf=-0.11309576,cover=285084.656\n',
 '0:[f0<610] yes=1,no=2,missing=2,gain=0.8125,cover=488908.188\n\t1:leaf=-0.112006716,cover=235173.766\n\t2:leaf=-0.111597531,cover=253734.422\n',
 '0:[f0<568] yes=1,no=2,missing=2,gain=0.9375,cover=447350.688\n\t1:leaf=-0.110721491,cover=208295.266\n\t2:leaf=-0.110281661,cover=239055.422\n',
 '0:[f0<610] yes=1,no=2,missing=2,gain=1.1875,cover=408864.875\n\t1:leaf=-0.109574139,cover=196599.844\n\t2:leaf=-0.109091453,cover=212265.016\n',
 '0:[f0<610] yes=1,no=2,missing=2,gain=1.40625,cover=373320.438\n\t1:leaf=-0.108559214,cover=179468.875\n\t2:leaf=-0.108032599,cover=193851.562\n',
 '0:[f0<610] yes=1,no=2,missing=2,gain=1.71875,cover=340569.469\n\t1:leaf=-0.107655361,cover=163683.812\n\t2:leaf=-0.107079498,cover=176885.641\n',
 '0:[f0<610] yes=1,no=2,missing=2,gain=1.90625,cover=310453.844\n\t1:leaf=-0.106848441,cover=149168.578\n\t2:leaf=-0.10

In [18]:
dmatrix = xgb.DMatrix(bank_txns[1][['amount']].values, label=bank_txns[1]['IsFraud'].values)
preds = model.predict(dmatrix)
np.mean((preds > 0.5) == bank_txns[1]['IsFraud'].values)

np.float64(0.9997127055247296)